In [1]:
%run nmt.py --cuda --mode test --test_src "./data/ch/test.txt" --test_tgt "./data/an/test.txt" --load_model "model.iter193500.bin" --save_to_file "result.txt" 

success read
success read


Namespace(batch_size=8, beam_size=5, clip_grad=5.0, cuda=True, debug=False, decode_max_time_step=200, dev_src=None, dev_tgt=None, dropout=0.0, embed_size=256, hidden_size=256, load_model='model.iter193500.bin', log_every=50, lr=0.001, lr_decay=0.5, max_niter=-1, mode='test', patience=5, raml_bias_groundtruth=False, raml_sample_file=None, raml_sample_mode='pre_sample', sample_method='random', sample_size=10, save_model_after=2, save_nbest=False, save_to='model', save_to_file='result.txt', seed=5783287, smooth_bleu=False, temp=0.85, test_src='./data/ch/test.txt', test_tgt='./data/an/test.txt', train_src=None, train_tgt=None, uniform_init=None, valid_metric='bleu', valid_niter=500, vocab=None)
load model from [model.iter193500.bin]


FileNotFoundError: [Errno 2] No such file or directory: 'model.iter193500.bin'

In [1]:
from __future__ import print_function

import os
import sys
import time
import argparse
from itertools import tee

import numpy as np

import torch
import torch.nn as nn
import torch.nn.utils
from torch.autograd import Variable
from torch import optim
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu

from util import read_corpus, data_iter
from vocab import Vocab, VocabEntry
from nmt import *


In [10]:
def interactive(path):
    assert path, 'You have to specify a pre-trained model'
    print('load model from [%s]' % path, file=sys.stderr)
    params = torch.load(path, map_location=lambda storage, loc: storage)
    vocab = params['vocab']
    saved_args = params['args']
    state_dict = params['state_dict']
    print(saved_args)
    model = NMT(saved_args, vocab)
    model.load_state_dict(state_dict)

    model.eval()
    model = model.cuda()

    src_sent = '它 的 西 北 方 有 座 山 。'
    src_sent = src_sent.strip().split(' ')
    hyps = model.translate(src_sent)
    print(hyps)
    for i, hyp in enumerate(hyps, 1):
        print('Hypothesis #%d: %s' % (i, ' '.join(hyp)))

In [9]:
interactive("ch-an_model/model.iter170000.bin")

load model from [ch-an_model/model.iter170000.bin]


Namespace(batch_size=16, beam_size=5, clip_grad=5.0, cuda=True, debug=False, decode_max_time_step=200, dev_src='./data/ch/valid.txt', dev_tgt='./data/an/valid.txt', dropout=0.0, embed_size=256, hidden_size=256, load_model=None, log_every=50, lr=0.001, lr_decay=0.5, max_niter=-1, mode='train', patience=5, raml_bias_groundtruth=False, raml_sample_file=None, raml_sample_mode='pre_sample', sample_method='random', sample_size=10, save_model_after=3, save_nbest=False, save_to='model', save_to_file=None, seed=5783287, smooth_bleu=False, temp=0.85, test_src='./data/ch/test.txt', test_tgt='./data/an/test.txt', train_src='./data/ch/train.txt', train_tgt='./data/an/train.txt', uniform_init=None, valid_metric='bleu', valid_niter=2000, vocab='vocab_ch_an.bin')
Hypothesis #1: <s> 西 北 有 座 。 </s>
Hypothesis #2: <s> 西 北 有 座 山 。 </s>
Hypothesis #3: <s> 其 西 北 有 座 。 </s>
Hypothesis #4: <s> 其 西 北 有 座 山 。 </s>
Hypothesis #5: <s> 西 北 有 山 。 </s>


/home/workspace/dual_learning/nmt/nmt.py:470: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_weight = F.softmax(att_weight)
/home/workspace/dual_learning/nmt/nmt.py:292: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  p_t = F.log_softmax(score_t)
